##### Don't forget to  set macOS settings to
##  Prevent automatic sleeping on power adaptor when display is off
#### before running this notebook

In [15]:
try:
    from googletrans import Translator
except:
    !python -m pip install googletrans==4.0.0-rc1
    from googletrans import Translator

import pandas as pd
import numpy as np
import os
import re
import time
from tqdm import tqdm
from beholder import print_methods, check_na


pd.set_option("display.max_colwidth", None)  # to display full text in columns
pd.set_option("display.max_columns", None)  # display all columns

In [2]:
df = pd.read_csv("/Users/velo1/SynologyDrive/GIT_syno/data/allDogDescriptions.csv")
# df.head(3)

## preprocessing

In [3]:
str_max_length = 5000   # max length of string to translate (Google API limit)

df_cropped = df[['age','description']]
# delete rows with NaN
df_cropped = df_cropped.dropna()
# delete duplicates
df_cropped.drop_duplicates(inplace=True)

def remove_noise(text)->str:
    '''Remove links, emails, and non-ASCII characters'''
    if isinstance(text, str):

        cleaned_text = re.sub(r'http\S+|www\.\S+', '', text)    # remove links
        cleaned_text = re.sub(r'\S+@\S+', '', cleaned_text)    # remove emails
        cleaned_text = re.sub(r'@', ' ', cleaned_text)      # remove @ (delimiter)
        cleaned_text = cleaned_text.encode("ascii", errors="ignore").decode()
        return cleaned_text
    else:
        return ""

df_cropped['description'] = df_cropped['description'].apply(remove_noise)


### The DELIMITER

In [4]:
# search for description field that contains "-"
delimiter = '@'
df_cropped.loc[df_cropped['description'].str.contains(delimiter)]


,age,description


In [5]:
# drop rows with description length > str_max_length
idx = df_cropped[df_cropped['description'].str.len() > str_max_length].index
df_cropped.drop(idx, inplace=True)
df_cropped.reset_index(drop=True, inplace=True)


In [6]:
#add 'translated' column to df_cropped
df_cropped['translated'] = ""

In [7]:
def concat_descriptions(df, start_ind=0, max_length=1000, delimiter = '~')->tuple:
    """
    Concatenate descriptions in df['description'] into one string
    with a delimiter between them.

    start_ind - index to start from
    max_length - max length of the string

    returns a tuple (string to translate, list of concateneted rows indexes)
    """
    
    prev_text = ""
    idx = []
    crop_tail = len(delimiter) + 1

    while start_ind < df.shape[0]:
        # retrieve text from df 
        text = df.loc[start_ind,'description']
        # if the value is a string and not empty
        if isinstance(text, str) and text != "":
            # if the length of the string is less than max_length
            if len(prev_text + text) < max_length:
                # concatenate the previous string with the current one and add a delimiter
                prev_text += text + " " + delimiter + " "
                # add the index of the row to the list
                idx.append(start_ind)
                start_ind += 1

                # if the index is the last one in the df
                if start_ind >= df.shape[0]:
                    # return the string and the list of indexes
                    return (prev_text[:-crop_tail], idx )
            else:
                return (prev_text[:-crop_tail], idx )

        else:
            start_ind += 1


In [8]:
translator = Translator()

## The LOOP

In [9]:
start_index = 0
error_cnt = 0
translated = ""
src_lang = "en" # source language src="auto"
dest_lang = "ru" # destination language
result = {"translated": 0, "dimension mismatch": 0, "translation error": 0}

total_rows = df_cropped.shape[0] 
pbar = tqdm(total=total_rows, desc="Translation Progress") 
while start_index < total_rows: 
    error_cnt = 0
    
    to_translate, rows = concat_descriptions(df_cropped, start_ind=start_index, max_length=str_max_length, delimiter=delimiter)
    while error_cnt < 5:
        try:
            translated = translator.translate(to_translate, dest=dest_lang, src=src_lang).text 
            list_ = translated.split(delimiter)
            if len(rows) == len (list_):
                df_cropped.loc[rows, 'translated'] = list_
                result["translated"] += len(rows)
            else:
                print(f"Translation dimension {len(list_)} mismatch  rows number {rows}")
                result["dimension mismatch"] += len(rows)
            start_index = rows[-1]+1
            pbar.update(len(rows))
            break
        except:
            error_cnt += 1
            start_index = rows[-1]+1
            # pbar.update(len(rows))
            time.sleep(.1)
        
    else:
        print(f"Translation error in rows {rows}")
        result["translation error"] += len(rows)
        pbar.update(len(rows))

        continue
pbar.close()
os.system('say "Translation finished"')


Translation Progress: 47680it [19:16:11,  2.52it/s]

Translation dimension 3 mismatch  rows number [45913, 45914, 45915, 45916]


Translation Progress: 47690it [19:16:12,  3.67it/s]

Translation dimension 9 mismatch  rows number [45917, 45918, 45919, 45920, 45921, 45922, 45923, 45924, 45925, 45926]


Translation Progress: 47861it [19:16:42,  6.36it/s]

Translation dimension 9 mismatch  rows number [46088, 46089, 46090, 46091, 46092, 46093, 46094, 46095, 46096, 46097]


Translation Progress: 48156it [19:17:42,  4.25it/s]

Translation dimension 6 mismatch  rows number [46386, 46387, 46388, 46389, 46390, 46391, 46392]


Translation Progress: 48181it [19:17:46,  1.44s/it]


0

In [22]:
df_cropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46418 entries, 0 to 46417
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   age          46418 non-null  object
 1   description  46418 non-null  object
 2   translated   46418 non-null  object
dtypes: object(3)
memory usage: 1.1+ MB


In [39]:
mask = df_cropped.translated.str.len() == 0 # mask for empty translation
df_cropped.drop(df_cropped[mask].index, inplace=True) # drop rows with empty translation
df_cropped.reset_index(drop=True, inplace=True) # reset index

In [18]:
df_cropped.tail(3)

,age,description,translated
46415,Adult,"Samson is a dog that will need someone to show him what love is. As soon as he was brought to us, he shut down-it took several days of our staff working with him to even get him out of his kennel and on a leash. He is one of those dogs, that when he loves you, he loves you. He trusts you. You have to prove yourself to him and he rewards you with the happiest of tail wags and devotion. It will take some work, but he is worth it!","Samson - собака, которой нужен кто -то, чтобы показать ему, что такое любовь.Как только его привезли к нам, он закрылся-ему потребовалось несколько дней наших сотрудников, работающих с ним, чтобы даже вытащить его из питомника и на поводке.Он одна из тех собак, что когда он любит тебя, он любит тебя.Он доверяет тебе.Вы должны доказать себя ему, и он вознаграждает вас самыми счастливыми из хвостовых вил и преданности.Это займет некоторую работу, но он того стоит!"
46416,Baby,"Buddy was an owner surrender by an older gentleman with an all too familiar story. Buddy was his daughter's dog, but she couldn't keep him anymore and decided Dad \""needed\"" a dog. He tried for a month but Buddy was just too much puppy for him. Buddy is your typical juvenile pup: needs more work on potty training, leash training is coming along, and needs work with his socializing with other dogs. He wants to jump and play and that isn't always what an older dog wants to do. He absolutely loves people and loves to go for walks with them.Have room for this goofy boy? Please come meet him!EDIT: 8/17/19Buddy spent 8 weeks in our Honor Dog Program at the Wyoming Honor Farm learning the AKC's Novice Basic Obedience training. He knows sit, stay, down, off and has learned socialization with other dogs and people; is crate, house, and leash trained.He is a wonderful boy that is ready for his people!","Buddy был владельцем сдачи старшим джентльменом с слишком знакомым историей.Бадди был собакой его дочери, но она больше не могла держать его и решила, что папа нужен \ »собака.Он пытался в течение месяца, но Бадди был для него слишком много щенка.Buddy - ваш типичный юношеский щенок: требует больше работы по тренировке горшок, обучение поводкам идет вместе, и нуждается в работе с его общением с другими собаками.Он хочет прыгать и играть, и это не всегда то, что хочет сделать старшая собака.Он абсолютно любит людей и любит ходить с ними.Пожалуйста, приходите познакомиться с ним! Редактировать: 8/17/19 Buddy провел 8 недель в нашей программе Honor Dog в Wyoming Honor Farm.Он знает, что сидит, оставайтесь, вниз, выключен и научился социализации с другими собаками и людьми;Ящик, дом и обучение поводка. Он замечательный мальчик, который готов к своим людям!"
46417,Adult,"Due to the small size of our volunteer base, we are unable to reply to inquiries to this listing. If you are serious about adopting, visit our website wbcrescue.org to read more about this dog and our adoption process, and to submit an application. Thanks for considering adoption!\n\nBeautiful Echo, like True, is a holdover from before WBCR closed in 2016. Like True, she remained with a foster family all this time. She is now ready and looking for her forever home, where she can be cherished. Echo is a special girl who will require a special family: she is fearful and reactive towards unfamiliar people, and like many female BCs, can be touchy with other dogs, although with careful management she peaceably cohabitates with several other dogs in her foster home. She loves her human friends and she is a total ball-a-holic. Shes got all kind of skills housetrained, crate-trained, basic manners and shes not a super high energy girl at all. Echo will require a savvy, experienced owner and a quiet situation, but we know the right family is out there for this glamorous girl!\n\nFor more info about Echo, to read about our adoption process, or to submit an application, visit wbcrescue.org","Из -за небольшого размера нашей волонтерской базы мы не м

In [14]:
df_cropped.to_csv("/Users/velo1/SynologyDrive/GIT_syno/data/allDogDescriptionsTranslated.csv", index=False)

#### Don't forget to  Restore macOS settings to
##  Perform automatic sleeping on power adaptor when display is off

ToDo:
1. do not drop cells with text > max_len but truncate them to max_len
